# HOER 02 - Data Standardization

In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

# After import you can jump to Step 2 if you wish to skip the data acquisition and use the data snapshot provided in the repo

## Step 1 - Acquire Data

### Wikipedia

In [2]:
# 1. Get the Wikipedia page (with User-Agent for robots policy)

url = "https://en.wikipedia.org/wiki/List_of_MPs_elected_in_the_2019_United_Kingdom_general_election"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
website_url = response.text

print(website_url[:300])    # Now should show real HTML, not robot policy

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 


In [3]:
# 2. Parse HTML and collect all tables

soup = BeautifulSoup(website_url, "html.parser")   # Parse HTML
tables = soup.find_all("table")                    # List of all <table> tags

print(f"Found {len(tables)} tables on the page.")

Found 12 tables on the page.


In [4]:
# 3. Find the MP table
#    First try phrase "Member returned"; if not found,
#    fall back to a table whose headers contain "Constituency" and "Member"

target_table = None

# Try by phrase
for table in tables:
    table_text = table.get_text(" ", strip=True)
    if "Member returned" in table_text:
        target_table = table
        break

# Fallback: by header structure
if target_table is None:
    for table in tables:
        th_texts = [th.get_text(strip=True) for th in table.find_all("th")]
        # Look for something that clearly looks like the MPs listing
        if "Constituency" in th_texts and any("Member" in h for h in th_texts):
            target_table = table
            break

if target_table is None:
    raise ValueError("Could not find a table with MP information.")

In [5]:
# 4. Extract headers and rows from the target table

# Get header cells
header_cells = target_table.find_all("th")
headers = [h.get_text(strip=True) for h in header_cells]

# Extract all data rows
rows = []
for tr in target_table.find_all("tr"):
    tds = tr.find_all("td")
    if not tds:
        continue
    row = [td.get_text(strip=True) for td in tds]
    rows.append(row)

# Make sure number of headers matches number of columns
max_cols = max(len(r) for r in rows) if rows else 0

if len(headers) > max_cols:
    headers = headers[:max_cols]
elif len(headers) < max_cols:
    headers = headers + [f"col_{i}" for i in range(len(headers), max_cols)]

In [6]:
# 5. Build the DataFrame

df_w = pd.DataFrame(rows, columns=headers)

df_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Constituency              650 non-null    object
 1   Affiliation of incumbent  650 non-null    object
 2   Member returned           650 non-null    object
 3   Notes                     650 non-null    object
 4                             650 non-null    object
 5   Affiliate                 650 non-null    object
 6   Source:[24]               650 non-null    object
dtypes: object(7)
memory usage: 35.7+ KB


In [7]:
df_w.head()

,Constituency,Affiliation of incumbent,Member returned,Notes,,Affiliate,Source:[24]
0,Aberavon,,Labour,Stephen Kinnock,,Labour,Seat held
1,Aberconwy,,Conservative,Robin Millar,,Conservative,"Incumbent,Guto Bebb, did not stand"
2,Aberdeen North,,Scottish National,Kirsty Blackman,,Scottish National,Seat held
3,Aberdeen South,,Conservative,Stephen Flynn,,Scottish National,"Incumbent,Ross Thomson, did not stand"
4,Airdrie and Shotts,,Scottish National,Neil Gray,,Scottish National,Seat held


In [8]:
# Remove unwanted columns and check length

df_w = df_w[['Constituency','Member returned','Notes']]

df_w.head()

,Constituency,Member returned,Notes
0,Aberavon,Labour,Stephen Kinnock
1,Aberconwy,Conservative,Robin Millar
2,Aberdeen North,Scottish National,Kirsty Blackman
3,Aberdeen South,Conservative,Stephen Flynn
4,Airdrie and Shotts,Scottish National,Neil Gray


In [9]:
len(df_w)

650

### They Work For You

In [10]:
# Download all current Members of Parliament

url = "https://www.theyworkforyou.com/mps/?f=csv"
df_t = pd.read_csv(url, header=0)

df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Person ID     650 non-null    int64 
 1   First name    650 non-null    object
 2   Last name     650 non-null    object
 3   Party         650 non-null    object
 4   Constituency  650 non-null    object
 5   URI           650 non-null    object
dtypes: int64(1), object(5)
memory usage: 30.6+ KB


In [11]:
df_t.head(n=5)

,Person ID,First name,Last name,Party,Constituency,URI
0,10001,Diane,Abbott,Independent,Hackney North and Stoke Newington,https://www.theyworkforyou.com/mp/10001/diane_...
1,26385,Jack,Abbott,Labour/Co-operative,Ipswich,https://www.theyworkforyou.com/mp/26385/jack_a...
2,25034,Debbie,Abrahams,Labour,Oldham East and Saddleworth,https://www.theyworkforyou.com/mp/25034/debbie...
3,26364,Shockat,Adam,Independent,Leicester South,https://www.theyworkforyou.com/mp/26364/shocka...
4,26483,Zubir,Ahmed,Labour,Glasgow South West,https://www.theyworkforyou.com/mp/26483/zubir_...


### Add facebook links

In [12]:
session = requests.Session()
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0.0.0 Safari/537.36"
    )
}

def facelink(uri):
    try:
        resp = session.get(uri, headers=headers, timeout=3)
        if resp.status_code != 200:
            return ""
    except:
        return ""

    html = resp.text
    match = re.search(r'https?://(?:www\.)?facebook\.com/[^\s"\'<>]+',
                      html, flags=re.IGNORECASE)
    if match:
        link = match.group(0)
        if link.lower() != "https://facebook.com":
            return link
    return ""

# EXECUÇÃO PARALELA (20 threads)
with ThreadPoolExecutor(max_workers=20) as executor:
    results = list(executor.map(facelink, df_t["URI"]))

df_t["Flink"] = results
df_t["FullName"] = df_t["First name"] + " " + df_t["Last name"]

# Mostrar os que têm Facebook
print(df_t[df_t["Flink"] != ""][["FullName", "Flink"]].head(20))


             FullName                                              Flink
0        Diane Abbott                   https://facebook.com/Dianeabbott
10    Heidi Alexander  https://facebook.com/Heidi-Alexander-MP-460236...
11      Rushanara Ali     https://facebook.com/Rushanara-Ali-37684209618
13  Rosena Allin-Khan                      https://facebook.com/DrRosena
20      Stuart Andrew            https://facebook.com/StuartAndrewPudsey
21   Tonia Antoniazzi                 https://facebook.com/ToniaForGower
28    Victoria Atkins            https://facebook.com/VoteVictoriaAtkins
39   Harriett Baldwin               https://facebook.com/harriettbaldwin
42      Steve Barclay                https://facebook.com/stevebarclaymp
53        Hilary Benn       https://facebook.com/hilarybenn4leedscentral
56        Clive Betts       https://facebook.com/clive.betts.sheffieldse
60       Bob Blackman           https://facebook.com/bobblackmanofficial
61    Kirsty Blackman                 https://faceb

### Saving to Local Storage

In [13]:
# Save file to local storage
# Note uncommenting these lines will overwrite the repo snapshot and may result 
# in different values in this and subsequent chapters

#df_t.to_csv('mps_they_raw.csv', index=False)
#df_w.to_csv('mps_wiki_raw.csv')

## Step 2 - Data Cleansing

In [14]:
# Theyworkforyou update their website after MP resignations, by-elections and general elections.

# To ensure consistency with the data at the time of writing the following statements overwrite
# the 'wiki' and 'they' raw dataframes with the snapshot provided in the repo

df_t = pd.read_csv('data/mps_they_raw.csv')
df_w = pd.read_csv('data/mps_wiki_raw.csv')

#### Wikipedia Data

In [15]:
df_w.head(n=5)

,Constituency,Member returned,Notes
0,NaN,NaN,NaN
1,Aberavon\n,Stephen Kinnock\n,Seat held\n
2,Aberconwy\n,Robin Millar\n,"Previous incumbent, Guto Bebb, did not stand\n"
3,Aberdeen North\n,Kirsty Blackman\n,Seat held\n
4,Aberdeen South\n,Stephen Flynn\n,"Previous incumbent, Ross Thomson, did not stand\n"


In [16]:
# Rename columns for consistency

df_w = df_w.rename(columns={ 'Member returned' : 'Fullname'})

# Remove null rows at start and end and strip tailing '\n'

df_w = df_w.dropna()
df_w['Constituency'] = df_w['Constituency'].str.rstrip("\n")
df_w['Fullname'] = df_w['Fullname'].str.rstrip("\n")

df_w.head(n=5)

,Constituency,Fullname,Notes
1,Aberavon,Stephen Kinnock,Seat held\n
2,Aberconwy,Robin Millar,"Previous incumbent, Guto Bebb, did not stand\n"
3,Aberdeen North,Kirsty Blackman,Seat held\n
4,Aberdeen South,Stephen Flynn,"Previous incumbent, Ross Thomson, did not stand\n"
5,Airdrie and Shotts,Neil Gray,Seat held\n


In [17]:
# Check \n not elsewhere in Fullname

df_w[df_w['Fullname'].astype(str).str.contains('\n')]

,Constituency,Fullname,Notes
228,Finchley and Golders Green,\nMike Freer,Seat held\n
373,Mole Valley,\nPaul Beresford,Seat held\n
545,Stockport,\nNavendu Mishra,"Previous incumbent, Ann Coffey, did not stand\n"
546,Stockton North,\nAlex Cunningham,Seat held\n
547,Stockton South,\nMatt Vickers,"Defeated incumbent, Paul Williams\n"
...,...,...,...
613,West Bromwich East,\nNicola Richards,"Previous incumbent, Tom Watson, did not stand\n"
614,West Bromwich West,\nShaun Bailey,"Previous incumbent, Adrian Bailey, did not sta..."
616,West Dunbartonshire,\nMartin Docherty-Hughes,Seat held\n
619,West Suffolk,\nMatthew Hancock,Seat held\n


In [18]:
# Strip leading '\n' 
df_w['Fullname'] = df_w['Fullname'].str.lstrip("\n")

# Check \n not elsewhere in Fullname
df_w[df_w['Fullname'].astype(str).str.contains('\n')]

,Constituency,Fullname,Notes


In [19]:
# Check \n not elsewhere in Constituency
df_w[df_w['Constituency'].astype(str).str.contains('\n')]

,Constituency,Fullname,Notes


In [20]:
df_w.head()

,Constituency,Fullname,Notes
1,Aberavon,Stephen Kinnock,Seat held\n
2,Aberconwy,Robin Millar,"Previous incumbent, Guto Bebb, did not stand\n"
3,Aberdeen North,Kirsty Blackman,Seat held\n
4,Aberdeen South,Stephen Flynn,"Previous incumbent, Ross Thomson, did not stand\n"
5,Airdrie and Shotts,Neil Gray,Seat held\n


In [21]:
# Split into Firstname and compound Lastname

df_w['Firstname'] = df_w['Fullname'].str.split().str[0]
df_w['Lastname'] = df_w['Fullname'].astype(str).apply(lambda x: ' '.join(x.split()[1:]))

df_w.head()

,Constituency,Fullname,Notes,Firstname,Lastname
1,Aberavon,Stephen Kinnock,Seat held\n,Stephen,Kinnock
2,Aberconwy,Robin Millar,"Previous incumbent, Guto Bebb, did not stand\n",Robin,Millar
3,Aberdeen North,Kirsty Blackman,Seat held\n,Kirsty,Blackman
4,Aberdeen South,Stephen Flynn,"Previous incumbent, Ross Thomson, did not stand\n",Stephen,Flynn
5,Airdrie and Shotts,Neil Gray,Seat held\n,Neil,Gray


In [22]:
# Check for compound lastnames

df_w[df_w['Lastname'].astype(str).str.contains(' ')]['Lastname']

31       de Cordova
134    Duncan Smith
393    Marie Morris
592      Ahmad Khan
Name: Lastname, dtype: object

In [23]:
df_w.head()

,Constituency,Fullname,Notes,Firstname,Lastname
1,Aberavon,Stephen Kinnock,Seat held\n,Stephen,Kinnock
2,Aberconwy,Robin Millar,"Previous incumbent, Guto Bebb, did not stand\n",Robin,Millar
3,Aberdeen North,Kirsty Blackman,Seat held\n,Kirsty,Blackman
4,Aberdeen South,Stephen Flynn,"Previous incumbent, Ross Thomson, did not stand\n",Stephen,Flynn
5,Airdrie and Shotts,Neil Gray,Seat held\n,Neil,Gray


#### They Work for You Data

In [24]:
df_t = df_t.rename(columns={'Last name' : 'Lastname', 'First name' : 'Firstname'})

df_t.head()

,Constituency,Firstname,Lastname,Flink
0,Hackney North and Stoke Newington,Diane,Abbott,https://facebook.com/Dianeabbott
1,Oldham East and Saddleworth,Debbie,Abrahams,NaN
2,Selby and Ainsty,Nigel,Adams,https://facebook.com/nigel.adamsmp
3,Hitchin and Harpenden,Bim,Afolami,NaN
4,Windsor,Adam,Afriyie,https://facebook.com/adamafriyieofficial


### Calculate Exact Match Counts 

In [25]:
len(df_w)

650

In [26]:
len(df_t)

650

In [27]:
# All matching columns for a merge (inner join) between two DataFrames: df_w and df_t.
# Merge only when all three columns match exactly in both DataFrames.
len(df_w.merge(df_t, on=['Constituency','Firstname','Lastname']))

599

In [28]:
# Merge only when 'Constituency' and 'Lastname' columns match exactly in both DataFrames.
len(df_w.merge(df_t, on=['Constituency','Lastname']))

607

In [29]:
# Merge only when First name and Last name columns match exactly in both DataFrames.
len(df_w.merge(df_t, on=['Firstname','Lastname'] ))

624

In [30]:
# Match on Consistency and Last name
len(df_w.merge(df_t, on=['Constituency','Lastname'] ))

607

In [31]:
# Match on Consistency and First name
len(df_w.merge(df_t, on=['Constituency','Firstname'] ))

602

In [32]:
# Match on Lastname
len(df_w.merge(df_t, on=['Lastname'] ))

982

In [33]:
# Match on Firstname
len(df_w.merge(df_t, on=['Firstname'] ))

2663

In [34]:
# Match on Consistency
len(df_w.merge(df_t, on=['Constituency'] ))

623

## Step 3 - Further Cleansing

### Constituency

In [35]:
# Perform an outer join on 'Constituency' and add a column showing the source of each row
df_w_outer = df_w.merge(df_t, on=['Constituency'],how="outer",indicator=True)

df_w_outer.head(3)

,Constituency,Fullname,Notes,Firstname_x,Lastname_x,Firstname_y,Lastname_y,Flink,_merge
0,Aberavon,Stephen Kinnock,Seat held\n,Stephen,Kinnock,Stephen,Kinnock,https://facebook.com/stephenkinnock,both
1,Aberconwy,Robin Millar,"Previous incumbent, Guto Bebb, did not stand\n",Robin,Millar,Robin,Millar,NaN,both
2,Aberdeen North,Kirsty Blackman,Seat held\n,Kirsty,Blackman,Kirsty,Blackman,https://facebook.com/aberdeennorth,both


In [36]:
# Show the first 5 constituencies that exist only in df_t (right side of the outer join)
df_w_outer[df_w_outer['_merge']=='right_only']['Constituency'].head(n=5)

55     Birmingham, Edgbaston
56     Birmingham, Erdington
57    Birmingham, Hall Green
58    Birmingham, Hodge Hill
59      Birmingham, Ladywood
Name: Constituency, dtype: object

In [37]:
# Show the first 5 constituencies that exist only in df_t (left side of the outer join)
df_w_outer[df_w_outer['_merge']=='left_only']['Constituency'].head(n=5)

46     Birmingham Edgbaston
47     Birmingham Erdington
48    Birmingham Hall Green
49    Birmingham Hodge Hill
50      Birmingham Ladywood
Name: Constituency, dtype: object

In [38]:
df_t.head(3)

,Constituency,Firstname,Lastname,Flink
0,Hackney North and Stoke Newington,Diane,Abbott,https://facebook.com/Dianeabbott
1,Oldham East and Saddleworth,Debbie,Abrahams,NaN
2,Selby and Ainsty,Nigel,Adams,https://facebook.com/nigel.adamsmp


In [39]:
# Remove commas from both dataframes

df_t['Constituency'] = df_t['Constituency'].str.replace(',', '')
df_w['Constituency'] = df_w['Constituency'].str.replace(',', '')

In [40]:
# Match on Consistency

len(df_w.merge(df_t, on=['Constituency']))

650

In [41]:
# Repeat perfect match count

len(df_w.merge(df_t, on=['Constituency','Firstname','Lastname']))

624

### Firstname

In [42]:
# Merge df_w and df_t on 'Constituency', keeping both versions of overlapping columns with suffixes
df_w_inner = df_w.merge(df_t, on=['Constituency'], suffixes=('_w', '_t'))

df_w_inner.head()

,Constituency,Fullname,Notes,Firstname_w,Lastname_w,Firstname_t,Lastname_t,Flink
0,Aberavon,Stephen Kinnock,Seat held\n,Stephen,Kinnock,Stephen,Kinnock,https://facebook.com/stephenkinnock
1,Aberconwy,Robin Millar,"Previous incumbent, Guto Bebb, did not stand\n",Robin,Millar,Robin,Millar,NaN
2,Aberdeen North,Kirsty Blackman,Seat held\n,Kirsty,Blackman,Kirsty,Blackman,https://facebook.com/aberdeennorth
3,Aberdeen South,Stephen Flynn,"Previous incumbent, Ross Thomson, did not stand\n",Stephen,Flynn,Stephen,Flynn,NaN
4,Airdrie and Shotts,Neil Gray,Seat held\n,Neil,Gray,Anum,Qaisar,NaN


In [43]:
# Filter rows where both the first names and last names differ between the two merged datasets
df_w_inner[(df_w_inner['Firstname_w'] != df_w_inner['Firstname_t']) &
           (df_w_inner['Lastname_w'] != df_w_inner['Lastname_t'])]

,Constituency,Fullname,Notes,Firstname_w,Lastname_w,Firstname_t,Lastname_t,Flink
4,Airdrie and Shotts,Neil Gray,Seat held\n,Neil,Gray,Anum,Qaisar,NaN
29,Batley and Spen,Tracy Brabin,Seat held\n,Tracy,Brabin,Kim,Leadbeater,NaN
47,Birmingham Erdington,Jack Dromey,Seat held\n,Jack,Dromey,Paulette,Hamilton,NaN
130,Chesham and Amersham,Cheryl Gillan,Seat held\n,Cheryl,Gillan,Sarah,Green,NaN
139,City of Chester,Chris Matheson,Seat held\n,Chris,Matheson,Samantha,Dixon,NaN
268,Hartlepool,Mike Hill,Seat held\n,Mike,Hill,Jill,Mortimer,NaN
392,Newton Abbot,Anne Marie Morris,Seat held\n,Anne,Marie Morris,Anne Marie,Morris,https://facebook.com/annemarie.morris.NA
410,North Shropshire,Owen Paterson,Seat held\n,Owen,Paterson,Helen,Morgan,NaN
432,Old Bexley and Sidcup,James Brokenshire,Seat held\n,James,Brokenshire,Louie,French,NaN
531,Southend West,David Amess,Seat held\n,David,Amess,Anna,Firth,NaN


In [44]:
df_w_inner.head(3)

,Constituency,Fullname,Notes,Firstname_w,Lastname_w,Firstname_t,Lastname_t,Flink
0,Aberavon,Stephen Kinnock,Seat held\n,Stephen,Kinnock,Stephen,Kinnock,https://facebook.com/stephenkinnock
1,Aberconwy,Robin Millar,"Previous incumbent, Guto Bebb, did not stand\n",Robin,Millar,Robin,Millar,NaN
2,Aberdeen North,Kirsty Blackman,Seat held\n,Kirsty,Blackman,Kirsty,Blackman,https://facebook.com/aberdeennorth


In [45]:
# Merge df_w and df_t on 'Constituency', appending suffixes to distinguish columns coming from each DataFrame
df_w_inner = df_w.merge(df_t, on=['Constituency'], suffixes=('_w', '_t'))

df_w_inner.head(3)

,Constituency,Fullname,Notes,Firstname_w,Lastname_w,Firstname_t,Lastname_t,Flink
0,Aberavon,Stephen Kinnock,Seat held\n,Stephen,Kinnock,Stephen,Kinnock,https://facebook.com/stephenkinnock
1,Aberconwy,Robin Millar,"Previous incumbent, Guto Bebb, did not stand\n",Robin,Millar,Robin,Millar,NaN
2,Aberdeen North,Kirsty Blackman,Seat held\n,Kirsty,Blackman,Kirsty,Blackman,https://facebook.com/aberdeennorth


In [46]:
# Filter rows where exactly one field (Firstname or Lastname) matches between df_w and df_t,
# meaning either Firstname matches but Lastname differs, or Lastname matches but Firstname differs
df_w_inner[
    (df_w_inner['Firstname_w'] == df_w_inner['Firstname_t']) & (df_w_inner['Lastname_w'] != df_w_inner['Lastname_t']) |
    (df_w_inner['Firstname_w'] != df_w_inner['Firstname_t']) & (df_w_inner['Lastname_w'] == df_w_inner['Lastname_t'])
]

,Constituency,Fullname,Notes,Firstname_w,Lastname_w,Firstname_t,Lastname_t,Flink
46,Birmingham Edgbaston,Preet Gill,Seat held\n,Preet,Gill,Preet Kaur,Gill,https://facebook.com/PreetKaurGillMP
99,Burton,Kate Griffiths,"Previous incumbent, Andrew Griffiths, did not ...",Kate,Griffiths,Kate,Kniveton,NaN
122,Central Suffolk and North Ipswich,Dan Poulter,Seat held\n,Dan,Poulter,Daniel,Poulter,NaN
272,Hayes and Harlington,John McDonnell,Seat held\n,John,McDonnell,John Martin,McDonnell,https://facebook.com/johnmcdonnellmp
311,Kingston upon Hull North,Diana Johnson,Seat held\n,Diana,Johnson,Diana R.,Johnson,https://facebook.com/DianaJohnsonHullNorth
316,Lagan Valley,Jeffrey Donaldson,Seat held\n,Jeffrey,Donaldson,Jeffrey M.,Donaldson,https://facebook.com/jeffrey.donaldson1
394,North Antrim,Ian Paisley,Seat held\n,Ian,Paisley,Ian,Paisley Jnr,NaN
502,Slough,Tanmanjeet Dhesi,Seat held\n,Tanmanjeet,Dhesi,Tan,Dhesi,https://facebook.com/tandhesi
510,South Down,Chris Hazzard,Seat held\n,Chris,Hazzard,Christopher,Hazzard,https://facebook.com/chris.hazzard.77
526,South West Norfolk,Liz Truss,Seat held\n,Liz,Truss,Elizabeth,Truss,https://facebook.com/ElizabethTrussSWNorfolk


In [47]:
df_t.head(3)

,Constituency,Firstname,Lastname,Flink
0,Hackney North and Stoke Newington,Diane,Abbott,https://facebook.com/Dianeabbott
1,Oldham East and Saddleworth,Debbie,Abrahams,NaN
2,Selby and Ainsty,Nigel,Adams,https://facebook.com/nigel.adamsmp


In [48]:
# Extract the first word from the 'Firstname' column in df_t (useful when names contain multiple parts)
df_t['Firstname'] = df_t['Firstname'].str.split().str[0]

df_t.head(3)

,Constituency,Firstname,Lastname,Flink
0,Hackney North and Stoke Newington,Diane,Abbott,https://facebook.com/Dianeabbott
1,Oldham East and Saddleworth,Debbie,Abrahams,NaN
2,Selby and Ainsty,Nigel,Adams,https://facebook.com/nigel.adamsmp


In [49]:
# Final resolved match count
df_resolved = df_w.merge(df_t, on=['Firstname','Lastname'] )
len(df_resolved)

628

In [50]:
# Merge df_w and df_t on 'Constituency', keeping duplicated column names separate using suffixes
df_w_inner = df_w.merge(df_t, on=['Constituency'], suffixes=('_w', '_t'))

# Select rows where exactly one of the name fields matches (Firstname or Lastname), 
# identifying partial mismatches between the two datasets
df_w_unmatched = df_w_inner[
    (df_w_inner['Firstname_w'] == df_w_inner['Firstname_t']) & (df_w_inner['Lastname_w'] != df_w_inner['Lastname_t']) |
    (df_w_inner['Firstname_w'] != df_w_inner['Firstname_t']) & (df_w_inner['Lastname_w'] == df_w_inner['Lastname_t'])
]

df_w_unmatched

,Constituency,Fullname,Notes,Firstname_w,Lastname_w,Firstname_t,Lastname_t,Flink
99,Burton,Kate Griffiths,"Previous incumbent, Andrew Griffiths, did not ...",Kate,Griffiths,Kate,Kniveton,NaN
122,Central Suffolk and North Ipswich,Dan Poulter,Seat held\n,Dan,Poulter,Daniel,Poulter,NaN
392,Newton Abbot,Anne Marie Morris,Seat held\n,Anne,Marie Morris,Anne,Morris,https://facebook.com/annemarie.morris.NA
394,North Antrim,Ian Paisley,Seat held\n,Ian,Paisley,Ian,Paisley Jnr,NaN
502,Slough,Tanmanjeet Dhesi,Seat held\n,Tanmanjeet,Dhesi,Tan,Dhesi,https://facebook.com/tandhesi
510,South Down,Chris Hazzard,Seat held\n,Chris,Hazzard,Christopher,Hazzard,https://facebook.com/chris.hazzard.77
526,South West Norfolk,Liz Truss,Seat held\n,Liz,Truss,Elizabeth,Truss,https://facebook.com/ElizabethTrussSWNorfolk
605,Wealden,Nus Ghani,Seat held\n,Nus,Ghani,Nusrat,Ghani,https://facebook.com/NusGhaniofficial
615,West Dunbartonshire,Martin Docherty-Hughes,Seat held\n,Martin,Docherty-Hughes,Martin,Docherty,https://facebook.com/MartinDochertySNP


## Sample Problem: Find MPs who held their seat and currently have Facebook account

In [51]:
df_resolved.head(n=5)

,Constituency_x,Fullname,Notes,Firstname,Lastname,Constituency_y,Flink
0,Aberavon,Stephen Kinnock,Seat held\n,Stephen,Kinnock,Aberavon,https://facebook.com/stephenkinnock
1,Aberconwy,Robin Millar,"Previous incumbent, Guto Bebb, did not stand\n",Robin,Millar,Aberconwy,NaN
2,Aberdeen North,Kirsty Blackman,Seat held\n,Kirsty,Blackman,Aberdeen North,https://facebook.com/aberdeennorth
3,Aberdeen South,Stephen Flynn,"Previous incumbent, Ross Thomson, did not stand\n",Stephen,Flynn,Aberdeen South,NaN
4,Aldershot,Leo Docherty,Seat held\n,Leo,Docherty,Aldershot,https://facebook.com/pg/LeoDocherty4Aldershot


In [52]:
# Select those records with a non-null Facebook reference

# Filter rows where 'Flink' is not empty and 'Notes' is exactly "Seat held\n"
df_heldwithface = df_resolved[(df_resolved['Flink']!="") & (df_resolved['Notes']=="Seat held\n")]
len(df_heldwithface)

474

## Save Files for Subsequent Chapters

In [53]:
# Save file to local storage
# Note uncommenting these lines will overwrite the repo snapshot and may result 
# in different values in this and subsequent chapters

# Save unmatched to pick up in Chapter 3

df_w_unmatched.to_csv('data/mps_unmatched.csv', index=False)

df_w.to_csv('data/mps_wiki_clean.csv', index=False)
df_t.to_csv('data/mps_they_clean.csv', index=False)